In [90]:
#!pip install opencv-python

In [91]:
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from random import randint
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split

In [92]:
camry_path = 'data/training/vehicle_images/camry_binary'
minivan_path = 'data/training/vehicle_images/minivan_binary'
suv_path = 'data/training/vehicle_images/suv_binary'

In [93]:
video = 'vid1.mp4'
vehicle_paths = [camry_path, minivan_path, suv_path]

In [94]:
training_images = []
labels = []

In [95]:
for path in vehicle_paths:
    for image in os.listdir(path):
        image = cv2.imread(os.path.join(path, image), cv2.IMREAD_GRAYSCALE)

        _, binary_image = cv2.threshold(image, 128, 1, cv2.THRESH_BINARY)  
        binary_image = cv2.resize(binary_image, (32,64))
        binary_image = binary_image.reshape(-1,1)

        training_images.append(binary_image)
        labels.append(1)
training_images = np.array(training_images)

In [96]:
blobs = np.array([np.array([randint(0,1) for _ in range(2048)]) for _ in range(len(training_images))])
for _ in blobs:
    labels.append(0)

In [97]:
labels = np.array(labels)
training_images = np.squeeze(training_images, axis=2)

In [98]:
x_train,X_test,y_train, y_test = train_test_split(np.concatenate([training_images, blobs]), labels, test_size=0.3,random_state=102)

In [99]:
model = Sequential()
model.add(Dense(2048, activation='sigmoid'))
model.add(Dense(2048, activation='relu'))
model.add(Dense(2048, activation='relu'))
model.add(Dense(2048, activation='relu'))
model.add(Dense(2048, activation='relu'))
model.add(Dense(1))

In [100]:
model.compile(optimizer='rmsprop', loss='mse')

In [101]:
model.fit(x=x_train, y=y_train, epochs=30)

Epoch 1/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 7s 203ms/step - loss: 41658.2695
Epoch 2/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 185ms/step - loss: 0.2643
Epoch 3/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 169ms/step - loss: 0.2678
Epoch 4/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 176ms/step - loss: 0.2283
Epoch 5/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 175ms/step - loss: 0.2058
Epoch 6/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 183ms/step - loss: 0.0974
Epoch 7/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 202ms/step - loss: 0.1028
Epoch 8/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 206ms/step - loss: 0.1826
Epoch 9/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 180ms/step - loss: 0.1624
Epoch 10/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 175ms/step - loss: 0.1047
Epoch 11/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 166ms/step - loss: 0.0388
Epoch 12/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 186ms/step - loss: 0.0634
Epoch 13/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 183ms/step - loss: 0.0610
Epoch 14/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 189ms/step - loss: 0.0963
Epoch 15/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 193ms/step - loss: 0.0194
Epoch 16/30
9/9

In [102]:
vid = cv2.VideoCapture(f'data/videos/{video}')
back_sub = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=50, detectShadows=False)
frame_width = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter('output_video.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 30, (frame_width, frame_height))

In [103]:
vid = cv2.VideoCapture(f'data/videos/{video}')
back_sub = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=50, detectShadows=False)
frame_width = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter('output_video.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 30, (frame_width, frame_height))

In [104]:
while (vid.isOpened()):
    # Read a frame from the video
    ret, frame = vid.read()
    if not ret:
        break

    # Apply the background subtractor to the frame
    fg_mask = back_sub.apply(frame)

    # Apply a binary threshold to clean up the mask
    _, binary = cv2.threshold(fg_mask, 200, 255, cv2.THRESH_BINARY)

    # Morphological operations to remove small noise
    kernel_open = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    kernel_close = cv2.getStructuringElement(cv2.MORPH_RECT, (15, 15))
    binary = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel_open, iterations=2)
    binary = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel_close, iterations=2)

    # Find contours to identify large moving objects (cars)
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Create a new mask to draw detected cars
    mask = np.zeros_like(binary)

    for contour in contours:
        # Filter out small contours based on area
        if cv2.contourArea(contour) > 500:  # Adjust this threshold as needed
            x, y, w, h = cv2.boundingRect(contour)
            obj = []
            for i in range(y, (y+h+1)):
                for j in range(x, (x+w+1)):
                    if [j,i] in contour:
                        obj.append(1)
                    else:
                        obj.append(0)
            
            obj = np.array(obj, dtype=np.uint8)
            obj = cv2.resize(obj, (32,64))
            obj = obj.reshape(1,-1)
            
            if model.predict(obj) < 0:
                continue
            
            cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 2)

    out.write(frame)
            


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 245ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━

In [105]:
out.release()